## Improve original dataset

In [9]:
import pandas as pd
import numpy as np
import re
import os

# ==========================================
# 1. CONFIGURATION
# ==========================================
# Paths
DATASET_1_PATH = "/home/phd2/Scrivania/CorsoRepo/embryo_valencia/datasets/DB_GV-rMII.xlsx"
DATASET_2_PATH = "/home/phd2/Scrivania/CorsoRepo/embryo_valencia/datasets/DB_MII_to_blasto.xlsx"
OUTPUT_PATH    = "/home/phd2/Scrivania/CorsoRepo/embryo_valencia/datasets/dataset_merged.csv"

# Target Columns in strict order
FINAL_COLUMNS = [
    "dish_well", 
    "BLASTO_NY", 
    "GV", 
    "sibling", 
    "timing_GVBD", 
    "timing_extrusion_PB", 
    "Note", 
    "start video", 
    "start frame", 
    "end video", 
    "end frame", 
    "NOTES", 
    "NOTES 1"
]

# ==========================================
# 2. HELPER FUNCTIONS
# ==========================================

def clean_pdb_col(val):
    """
    Splits 'D2016...pdb GV-rMII' into ('D2016...', 'GV-rMII')
    """
    if pd.isna(val): return "", ""
    s = str(val).strip()
    if ".pdb" in s.lower():
        idx = s.lower().find(".pdb") + 4
        name = s[:idx].strip()
        stage = s[idx:].strip()
        clean_name = re.sub(r'\.pdb$', '', name, flags=re.IGNORECASE)
        return clean_name, stage
    return s, ""

def map_stage_to_code(stage_str):
    """Maps stage string (e.g. GV-rMII) to numeric GV code."""
    s = str(stage_str).upper().replace("-", "").replace(".", "")
    if "RMII" in s or "MII" in s: return 2
    if "MI" in s: return 1
    if "GV" in s: return 0
    return np.nan

def get_ivm_note(gv_code):
    """Maps numeric code to class label."""
    if gv_code == 0: return "GV_to_GV"
    elif gv_code == 1: return "GV_to_M1"
    elif gv_code == 2: return "GV_to_M2"
    return "Unknown_IVM"

def normalize_colnames(df):
    """
    Robust column renaming. 
    Matches fuzzy names (e.g. 'starti video') to standard keys.
    """
    col_map = {}
    for c in df.columns:
        cl = str(c).lower().strip()
        
        # Mapping rules
        if "notes 1" in cl: col_map[c] = "NOTES 1"
        elif "notes" in cl: col_map[c] = "NOTES"
        elif "start" in cl and "video" in cl: col_map[c] = "start video"
        elif "start" in cl and "frame" in cl: col_map[c] = "start frame"
        elif "end" in cl and "video" in cl: col_map[c] = "end video"
        elif "end" in cl and "frame" in cl: col_map[c] = "end frame"
        elif "gvbd" in cl: col_map[c] = "timing_GVBD"
        elif "pb" in cl and "timing" in cl: col_map[c] = "timing_extrusion_PB"
        elif "blasto" in cl: col_map[c] = "BLASTO_NY"
        # Preserve specific 'well' column, avoid matching 'dish_well'
        elif "well" in cl and "dish" not in cl: col_map[c] = "well" 
        
    return df.rename(columns=col_map)

def clean_id_from_db2(val):
    """
    Cleans IDs from DB2 which often look like 'D2016..._wells_1'.
    Returns 'D2016..._1'.
    """
    s = str(val).strip()
    return re.sub(r'_wells_', '_', s, flags=re.IGNORECASE).strip()

# ==========================================
# 3. PROCESSING LOGIC
# ==========================================

dfs = []

print("--- Starting Processing ---")

# ---------------------------------------------------------
# 1. DATASET 1: SHEET 1 (GV / IVM)
# ---------------------------------------------------------
print("1. Processing DB_GV-rMII.xlsx (Sheet 1: GVs)...")
try:
    d1s1 = pd.read_excel(DATASET_1_PATH, sheet_name=0)
    # Check if header is on row 0 or 1
    if 'well' not in [str(c).lower() for c in d1s1.columns]:
        d1s1 = pd.read_excel(DATASET_1_PATH, sheet_name=0, header=1)
    
    d1s1 = normalize_colnames(d1s1)
    
    # Find PDB column (contains .pdb)
    pdb_col = next((c for c in d1s1.columns if d1s1[c].astype(str).str.contains(r'\.pdb', case=False).any()), d1s1.columns[0])
    
    clean_rows = []
    for idx, row in d1s1.iterrows():
        name, stage = clean_pdb_col(row[pdb_col])
        gv_val = map_stage_to_code(stage)
        note = get_ivm_note(gv_val)
        well = str(row.get('well', '')).split('.')[0].strip()
        dish_well = f"{name}_{well}"
        
        new_row = {
            "dish_well": dish_well,
            "BLASTO_NY": pd.to_numeric(row.get('BLASTO_NY'), errors='coerce'),
            "GV": gv_val,
            "sibling": 0,
            "timing_GVBD": row.get('timing_GVBD', np.nan),
            "timing_extrusion_PB": row.get('timing_extrusion_PB', np.nan),
            "Note": note,
            "start video": row.get('start video', np.nan),
            "start frame": row.get('start frame', np.nan),
            "end video": row.get('end video', np.nan),
            "end frame": row.get('end frame', np.nan),
            "NOTES": row.get('NOTES', ""),
            "NOTES 1": row.get('NOTES 1', "")
        }
        clean_rows.append(new_row)
    
    dfs.append(pd.DataFrame(clean_rows))
    print(f"   -> Added {len(clean_rows)} GVs.")

except Exception as e:
    print(f"   Error D1S1: {e}")

# ---------------------------------------------------------
# 2. DATASET 2: SHEET 1 (Rescue MII)
# ---------------------------------------------------------
print("2. Processing DB_MII_to_blasto.xlsx (Sheet 1: Rescue MII)...")
try:
    # Sheet 0 (first sheet) is usually Rescue MII (rM2)
    d2s1 = pd.read_excel(DATASET_2_PATH, sheet_name=0)
    d2s1 = normalize_colnames(d2s1)
    
    pdb_col = next((c for c in d2s1.columns if 'pdb' in str(c).lower()), d2s1.columns[0])
    
    clean_rows = []
    for idx, row in d2s1.iterrows():
        dish_well = clean_id_from_db2(row[pdb_col])
        blasto = pd.to_numeric(row.get('BLASTO_NY'), errors='coerce')
        # Logic: rM2
        note = "rM2_to_blasto" if blasto == 1 else "rM2_to_no_blasto"
        
        new_row = {
            "dish_well": dish_well,
            "BLASTO_NY": blasto,
            "GV": 3, # Rescue MII
            "sibling": 0,
            "timing_GVBD": np.nan,
            "timing_extrusion_PB": np.nan,
            "Note": note,
            "start video": row.get('start video', np.nan),
            "start frame": row.get('start frame', np.nan),
            "end video": row.get('end video', np.nan),
            "end frame": row.get('end frame', np.nan),
            "NOTES": row.get('NOTES', ""),
            "NOTES 1": row.get('NOTES 1', "")
        }
        clean_rows.append(new_row)
        
    dfs.append(pd.DataFrame(clean_rows))
    print(f"   -> Added {len(clean_rows)} Rescue MIIs.")

except Exception as e:
    print(f"   Error D2S1: {e}")

# ---------------------------------------------------------
# 3. DATASET 2: SHEET 2 (Native MII)
# ---------------------------------------------------------
print("3. Processing DB_MII_to_blasto.xlsx (Sheet 2: Native MII)...")
try:
    # Sheet 1 (second sheet) is Native MII (M2)
    d2s2 = pd.read_excel(DATASET_2_PATH, sheet_name=1)
    d2s2 = normalize_colnames(d2s2)
    
    pdb_col = next((c for c in d2s2.columns if 'pdb' in str(c).lower()), d2s2.columns[0])
    
    clean_rows = []
    for idx, row in d2s2.iterrows():
        dish_well = clean_id_from_db2(row[pdb_col])
        blasto = pd.to_numeric(row.get('BLASTO_NY'), errors='coerce')
        # Logic: Native M2
        note = "M2_to_blasto" if blasto == 1 else "M2_to_no_blasto"
        
        new_row = {
            "dish_well": dish_well,
            "BLASTO_NY": blasto,
            "GV": -1, # Native MII
            "sibling": 1,
            "timing_GVBD": np.nan,
            "timing_extrusion_PB": np.nan,
            "Note": note,
            "start video": row.get('start video', np.nan),
            "start frame": row.get('start frame', np.nan),
            "end video": row.get('end video', np.nan),
            "end frame": row.get('end frame', np.nan),
            "NOTES": row.get('NOTES', ""),
            "NOTES 1": row.get('NOTES 1', "")
        }
        clean_rows.append(new_row)
        
    dfs.append(pd.DataFrame(clean_rows))
    print(f"   -> Added {len(clean_rows)} Native MIIs.")

except Exception as e:
    print(f"   Error D2S2: {e}")

# ==========================================
# 4. EXPORT
# ==========================================
if dfs:
    df_total = pd.concat(dfs, ignore_index=True)
    
    # 1. Filter out empty/short IDs
    df_total = df_total[df_total['dish_well'].astype(str).str.len() > 5]
    
    # 2. Dedup (Keep first occurrence)
    before_len = len(df_total)
    df_total.drop_duplicates(subset='dish_well', keep='first', inplace=True)
    
    # 3. Ensure all columns exist and fill NaNs
    for c in FINAL_COLUMNS:
        if c not in df_total.columns:
            df_total[c] = np.nan
            
    df_total = df_total[FINAL_COLUMNS]
    
    # 4. Save
    os.makedirs(os.path.dirname(OUTPUT_PATH), exist_ok=True)
    df_total.to_csv(OUTPUT_PATH, index=False)
    
    print("\n=== SUCCESS ===")
    print(f"Total Rows Processed: {before_len}")
    print(f"Final Rows (after dedup): {len(df_total)}")
    print(f"File saved to: {OUTPUT_PATH}")
    print("\nClass Distribution:")
    print(df_total['Note'].value_counts())
else:
    print("\nNo data processed.")

--- Starting Processing ---
1. Processing DB_GV-rMII.xlsx (Sheet 1: GVs)...
   -> Added 99 GVs.
2. Processing DB_MII_to_blasto.xlsx (Sheet 1: Rescue MII)...
   -> Added 51 Rescue MIIs.
3. Processing DB_MII_to_blasto.xlsx (Sheet 2: Native MII)...
   -> Added 60 Native MIIs.

=== SUCCESS ===
Total Rows Processed: 207
Final Rows (after dedup): 206
File saved to: /home/phd2/Scrivania/CorsoRepo/embryo_valencia/datasets/dataset_merged.csv

Class Distribution:
Note
GV_to_M2            47
rM2_to_no_blasto    42
M2_to_blasto        39
GV_to_GV            36
M2_to_no_blasto     21
GV_to_M1            12
rM2_to_blasto        9
Name: count, dtype: int64


In [13]:
# Select and create dataset_final_merged.csv from the dataset_merged selecting only the raws that have empty columns "NOTES" and "NOTES 1" 
df_merged = pd.read_csv("/home/phd2/Scrivania/CorsoRepo/embryo_valencia/datasets/dataset_merged.csv")
df_final = df_merged[
    (df_merged["NOTES"].isna() | (df_merged["NOTES"].astype(str).str.strip() == "")) &
    (df_merged["NOTES 1"].isna() | (df_merged["NOTES 1"].astype(str).str.strip() == ""))
]
df_final = df_final.drop(columns=["NOTES", "NOTES 1"])
df_final.to_csv("/home/phd2/Scrivania/CorsoRepo/embryo_valencia/datasets/dataset_final_merged2.csv", index=False)

In [10]:
print("\nPreview:")
print(df_total.iloc[10:20])


Preview:
                        dish_well  BLASTO_NY   GV  sibling timing_GVBD  \
10  D2019.02.15_S012161_I0631_D_1        0.0  2.0        0         0.6   
11  D2019.02.15_S012161_I0631_D_2        0.0  2.0        0         5.6   
12  D2019.02.15_S012161_I0631_D_5        1.0  2.0        0         4.6   
13  D2019.02.15_S012161_I0631_D_6        0.0  2.0        0           -   
14   D2019.03.14_S00126_I0758_D_3        0.0  2.0        0         7.1   
15   D2019.03.14_S00126_I0758_D_6        0.0  2.0        0         9.6   
16   D2019.03.14_S00126_I0758_D_7        0.0  2.0        0         9.2   
17   D2019.05.26_S01883_I0406_D_9        0.0  2.0        0         1.4   
18  D2019.05.26_S01883_I0406_D_10        0.0  2.0        0         2.4   
19  D2019.05.26_S01883_I0406_D_11        0.0  2.0        0         0.9   

   timing_extrusion_PB      Note start video  start frame  end video  \
10                13.8  GV_to_M2         0.1          1.0       24.3   
11                20.8  GV_to_M